Tests for state dependency, which we define as $\Delta F/F$ after the perturbation which encodes behavior before, for a particular behavior vs. all other behaviors. When testing, we include dependence in our model on specifici behaviors after the manipulation, to control for correlation between before and after behaviors.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#%matplotlib qt

In [3]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io

from keller_zlatic_vnc.data_processing import count_unique_subjs_per_transition
from keller_zlatic_vnc.data_processing import extract_transitions
from keller_zlatic_vnc.data_processing import generate_transition_dff_table
from keller_zlatic_vnc.data_processing import read_raw_transitions_from_excel
from keller_zlatic_vnc.data_processing import recode_beh
from keller_zlatic_vnc.linear_modeling import one_hot_from_table
from keller_zlatic_vnc.linear_modeling import order_and_color_interaction_terms
from keller_zlatic_vnc.linear_modeling import reference_one_hot_to_beh


from janelia_core.stats.regression import grouped_linear_regression_ols_estimator
from janelia_core.stats.regression import grouped_linear_regression_acm_stats
from janelia_core.stats.regression import visualize_coefficient_stats

/Users/williambishop/Documents/Janelia_Research/Projects/janelia_core/janelia_core/fileio/exp_reader.py:21: UserWarning: Unable to locate pyklb module.  Will not be able to read in .klb files.
  warnings.warn('Unable to locate pyklb module.  Will not be able to read in .klb files.')


In [4]:
plt.rc('font', family='arial', weight='normal', size=15)

## Options for analysis

In [5]:
# Type of cells we fit models to
cell_type = 'handle' 

# If we fit data to perturbations targeted at 'A4', 'A9' or 'both'
manip_type = 'both'

# Define the cutoff time we use to define quiet behaviors following stimulation
cut_off_time = 3.656#3.656 #9.0034

# Behavior that activity after the stimulus is referened to
after_beh_ref = 'F'

## Location of the data

In [6]:
data_folder = r'/Users/williambishop/Desktop/extracted_dff_v2'
transition_file = 'transition_list.xlsx'

a00c_a4_act_data_file = 'A00c_activity_A4.mat'
a00c_a9_act_data_file = 'A00c_activity_A9.mat'

basin_a4_act_data_file = 'Basin_activity_A4.mat'
basin_a9_act_data_file = 'Basin_activity_A9.mat'

handle_a4_act_data_file = 'Handle_activity_A4.mat'
handle_a9_act_data_file = 'Handle_activity_A9.mat'

## Specify some parameters we use in the code below

In [7]:
if cell_type == 'a00c':
    a4_act_file = a00c_a4_act_data_file
    a9_act_file = a00c_a9_act_data_file
elif cell_type == 'basin':
    a4_act_file = basin_a4_act_data_file
    a9_act_file = basin_a9_act_data_file
elif cell_type == 'handle':
    a4_act_file = handle_a4_act_data_file
    a9_act_file = handle_a9_act_data_file
else:
    raise(ValueError('The cell type ' + cell_type + ' is not recogonized.'))

## Load data

In [8]:
# Read in raw transitions
raw_trans = read_raw_transitions_from_excel(pathlib.Path(data_folder) / transition_file)

# Read in activity
a4_act = scipy.io.loadmat(pathlib.Path(data_folder) / a4_act_file, squeeze_me=True)
a9_act = scipy.io.loadmat(pathlib.Path(data_folder) / a9_act_file, squeeze_me=True)

# Correct mistake in labeling if we need to
if cell_type == 'basin' or cell_type == 'handle':
    ind = np.argwhere(a4_act['newTransitions'] == '0824L2CL')[1][0]
    a4_act['newTransitions'][ind] = '0824L2-2CL'

# Recode behavioral annotations
raw_trans = recode_beh(raw_trans, 'Beh Before')
raw_trans = recode_beh(raw_trans, 'Beh After')

# Extract transitions
trans = extract_transitions(raw_trans, cut_off_time)

# Generate table of data 
a4table = generate_transition_dff_table(act_data=a4_act, trans=trans)
a9table = generate_transition_dff_table(act_data=a9_act, trans=trans)

# Put the tables together
a4table['man_tgt'] = 'A4'
a9table['man_tgt'] = 'A9'
data = a4table.append(a9table, ignore_index=True)

## Down select for manipulation target

In [9]:
if manip_type == 'A4' or manip_type == 'a4':
    print('Analyzing only A4 manipulation events.')
    data = data[data['man_tgt'] == 'A4']
elif manip_type == 'A9' or manip_type == 'a9':
    print('Analyzing only A9 manipulation events.')
    data = data[data['man_tgt'] == 'A9']
else:
    print('Analyzing all manipulation events.')

Analyzing all manipulation events.


## Determine which behaviors are present before and after the manipulation

In [10]:
trans_subj_cnts = count_unique_subjs_per_transition(data)

In [11]:
data

,subject_id,cell_id,event_id,beh_before,beh_after,dff_before,dff_during,dff_after,man_tgt
0,CW_17-08-23-L2,11.0,0,F,B,0.005696,0.274746,0.589443,A4
1,CW_17-08-23-L2,11.0,1,F,B,-0.004324,0.108657,0.383802,A4
2,CW_17-08-23-L2,11.0,2,F,B,-0.023979,0.152021,0.332981,A4
3,CW_17-08-23-L2,11.0,0,F,B,-0.001543,0.515289,0.923596,A4
4,CW_17-08-23-L2,11.0,1,F,B,0.017293,0.119278,0.294427,A4
...,...,...,...,...,...,...,...,...,...
3208,CW_17-11-08-L3,4.0,2,Q,F,0.028811,0.403325,0.842192,A9
3209,CW_17-11-08-L3,4.0,3,Q,F,0.006838,0.398595,0.730813,A9
3210,CW_17-11-08-L3,4.0,4,Q,F,0.007750,0.222275,0.528680,A9
3211,CW_17-11-08-L3,4.0,5,Q,P,0.002588,0.208364,0.350184,A9


In [38]:
before_keep_rows = data['beh_before'].apply(lambda x: x in set(['F', 'Q']))
after_keep_rows = data['beh_after'].apply(lambda x: x in set(['P', 'B']))

In [39]:
data[after_keep_rows & before_keep_rows]

,subject_id,cell_id,event_id,beh_before,beh_after,dff_before,dff_during,dff_after,man_tgt
0,CW_17-08-23-L2,11.0,0,F,B,0.005696,0.274746,0.589443,A4
1,CW_17-08-23-L2,11.0,1,F,B,-0.004324,0.108657,0.383802,A4
2,CW_17-08-23-L2,11.0,2,F,B,-0.023979,0.152021,0.332981,A4
3,CW_17-08-23-L2,11.0,0,F,B,-0.001543,0.515289,0.923596,A4
4,CW_17-08-23-L2,11.0,1,F,B,0.017293,0.119278,0.294427,A4
...,...,...,...,...,...,...,...,...,...
3183,CW_17-11-08-L3,1.0,5,Q,P,0.009342,0.092434,0.250846,A9
3190,CW_17-11-08-L3,5.0,5,Q,P,0.018738,0.061997,0.054916,A9
3197,CW_17-11-08-L3,8.0,5,Q,P,-0.007937,0.485015,0.264182,A9
3204,CW_17-11-08-L3,4.0,5,Q,P,0.001380,0.132986,0.171090,A9


In [ ]:
after_beh_sum = trans_subj_cnts.sum()
after_behs = [b for b in after_beh_sum[after_beh_sum > 0].index]

before_beh_sum = trans_subj_cnts.sum(1)
before_behs = [b for b in before_beh_sum[before_beh_sum > 0].index]

## Pull out $\Delta F/F$

In [ ]:
dff = data['dff_after'].to_numpy()

## Find grouping of data by subject

In [ ]:
unique_ids = data['subject_id'].unique()
g = np.zeros(len(data))
for u_i, u_id in enumerate(unique_ids):
    g[data['subject_id'] == u_id] = u_i

## Fit models and calculate stats

In [ ]:
after_behs_ref = list(set(after_behs).difference(after_beh_ref))

In [ ]:
n_before_behs = len(before_behs)
before_betas = np.zeros(n_before_behs)
before_c_ints = np.zeros([2, n_before_behs])
before_sig = np.zeros(n_before_behs, dtype=np.bool)
for b_i, b in enumerate(before_behs):
    one_hot_data_ref, one_hot_vars_ref = one_hot_from_table(data, beh_before=[b], beh_after=after_behs_ref)
    one_hot_data_ref = np.concatenate([one_hot_data_ref, np.ones([one_hot_data_ref.shape[0], 1])], axis=1)
    one_hot_vars_ref = one_hot_vars_ref + ['ref'] 
    
    _, v, _ = np.linalg.svd(one_hot_data_ref)
    if np.min(v) < .001:
        raise(RuntimeError('regressors are nearly co-linear'))
        
    beta, acm, n_gprs = grouped_linear_regression_ols_estimator(x=one_hot_data_ref, y=dff, g=g)
    stats = grouped_linear_regression_acm_stats(beta=beta, acm=acm, n_grps=n_gprs, alpha=.05)
    
    before_betas[b_i] = beta[0]
    before_c_ints[:, b_i] = stats['c_ints'][:,0]
    before_sig[b_i] = stats['non_zero'][0]

## Visualize stats

In [ ]:
visualize_coefficient_stats(var_strs=before_behs, theta=before_betas, c_ints=before_c_ints, sig=before_sig,
                            x_axis_rot=0)
plt.ylabel('$\Delta F / F$')
plt.xlabel('Behavior Before Manipulation')
plt.tight_layout()
fig = plt.gcf()
fig.set_size_inches(8, 6)